In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import _init_paths
import os
import sys
import numpy as np
import argparse
import pprint
import pdb
import time
import cv2
import torch
from torch.autograd import Variable
import torch.optim as optim

import torchvision.transforms as transforms
from torch.utils.data.sampler import Sampler

from roi_data_layer.roidb import combined_roidb, rank_roidb_ratio, filter_roidb
from roi_data_layer.roibatchLoader import roibatchLoader
from model.utils.config import cfg, cfg_from_file, cfg_from_list, get_output_dir
from model.utils.net_utils import weights_normal_init, save_net, load_net, \
    adjust_learning_rate, save_checkpoint, clip_gradient



import pickle

In [2]:
import FedUtils

In [3]:
class Arguments():
    def __init__(self):
        self.class_agnostic =False
        self.epochs = 3
        self.lr = 0.001
        self.lr_decay_step = 4
        self.dataset = 'KAIST'
        self.net = 'vgg16'
        self.batch_size = 24
        self.nw = 2
        self.cuda = True
        #self.round = 10
        self.mGPUs=True
        self.optimizer ="sgd"
        self.k=5
        #self.device = th.device("cpu")
        
args = Arguments()

In [4]:
parties = 3
round_i = 4
#model_path = ['models/vgg16/KAIST/faster_rcnn_KAIST_3AVG/faster_rcnn_KAIST_campus_1_3_718.pth',
#'models/vgg16/KAIST/faster_rcnn_KAIST_3AVG/faster_rcnn_KAIST_road_1_3_513.pth',
#'models/vgg16/KAIST/faster_rcnn_KAIST_3AVG/faster_rcnn_KAIST_downtown_1_3_566.pth']
model_path = ['models/vgg16/KAIST/wkAVG/faster_rcnn_KAIST_campus_'+str(round_i)+'_3_718.pth',
'models/vgg16/KAIST/wkAVG/faster_rcnn_KAIST_road_'+str(round_i)+'_3_513.pth',
'models/vgg16/KAIST/wkAVG/faster_rcnn_KAIST_downtown_'+str(round_i)+'_3_566.pth']


imdb_classes =  ('__background__',  # always index 0
                          'person',
                          'people','cyclist'
                         )

model_list=[None] * parties
for i in range(parties):
    model_list[i],optimizer = FedUtils.load_model(imdb_classes,model_path[i], args,cfg)
    





Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth
Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth
Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth


### save test image

In [24]:
def preproc_frcnn(img):
    """ follow the processing steps of fasterRCNN
    """
    img = np.asarray(img[:,:,::-1], dtype=np.float32)
    PIXEL_MEANS = np.array([[[102.9801, 115.9465, 122.7717]]])
    img -= PIXEL_MEANS
    
    
    return torch.tensor(img)

In [26]:
from skimage.io import imread

BASE_DIR = '/home/superorange5/data/kaist_test/kaist_test_visible'
file_names = [f for f in os.listdir(BASE_DIR) if '.png' in f]
#scene_tags = [scene_tag(f) for f in file_names]

imgs_raw = [imread(os.path.join(BASE_DIR, fn)) for fn in file_names]
imgs = [preproc_frcnn(img).unsqueeze(0).permute(0, 3, 1, 2) for img in imgs_raw]

### load test image

In [8]:
testimg_pickle_path = 'testimg2252.pkl'


In [9]:
# with open(testimg_pickle_path, 'wb') as handle:
#     pickle.dump(imgs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
with open(testimg_pickle_path, 'rb') as handle:
    test_images = pickle.load(handle)

In [15]:
from frcnn_helper import *


        
def getWeight(test_images,model_list, args):
    
    wk_list = []
    for fasterRCNN in model_list:
        if args.mGPUs:
            fasterRCNN = fasterRCNN.module
        X = get_features(fasterRCNN, test_images, args.batch_size)/255.0
        wk_value = within_cluster_dispersion(X, n_cluster=args.k)
        wk_list.append(wk_value)
        print(wk_value)
    
    return wk_list

In [5]:
print(round_i)

4


In [20]:

#i=1

#wk_list_prev = [1.792,2.595,2.133]
#wk_list_prev=[1.8834246677171604, 2.603966431543569, 1.8537786606301114]  #round1
#wk_list_prev=[3.0300931747670314,4.143017952287369,3.614508784792944]  # round2
wk_list_prev=[2.6206474483306246,3.016229260855767,2.688841387] #round3


    
with open(testimg_pickle_path, 'rb') as handle:
    test_images = pickle.load(handle)
# get within class dispersion        

wk_list_curr = getWeight(test_images,model_list, args)
if round_i==1:
    wk_diff = wk_list_curr
else:
    wk_diff=[]
    for list1_c, list2_p in zip(wk_list_prev, wk_list_curr):        
        wk_diff.append(list1_c-list2_p)

print('diff={}'.format(wk_diff))

wk_ratio = softmax(wk_diff).tolist()    
print('wk_ratio={}'.format(wk_ratio))
wk_list_prev = wk_list_curr

3.0300931747670314
4.143017952287369
3.614508784792944


### test softmax

In [14]:
wk_list1=[1.8834246677171604, 2.603966431543569, 1.8537786606301114]  #round1
wk_list2=[3.0300931747670314,4.143017952287369,3.614508784792944]  # round2
wk_list3=[2.6206474483306246,3.016229260855767,2.688841387] #round3

wk_diff=[]
for list1_c, list2_p in zip(wk_list3, wk_list2):        
    wk_diff.append(list1_c-list2_p)
print('diff={}'.format(wk_diff))
from scipy.special import softmax
I_ratio = softmax(wk_diff).tolist()


diff=[-0.40944572643640686, -1.1267886914316025, -0.9256673977929437]


### avg by weight

In [7]:
#wk_ratio = [0.273744286,0.378889121,0.347366593] #wk_value_ratio round=2
#wk_ratio = [0.273369509,0.384206358,0.342424133] #wk_value_ratio round=3
#wk_ratio = [0.285985,0.394200378,0.319814622] #wk_value_ratio round=4

#wk_ratio = [0.295141897,0.315678326,0.389179776]  #wk_diff_ratio round=1

#wk_ratio = [0.239251713,0.48078381,0.279964477]  #wk_diff_ratio round=2
#wk_ratio = [0.275741057,0.350559489,0.373699454] #wk_diff_ratio round=3
wk_ratio = [0.229953847,0.349812376,0.420233777] #wk_diff_ratio round=4


#wk_ratio = [0.16262228, 0.48690203, 0.35047569]  #softmax(wk_value) round=2
#wk_ratio = [0.18246407,0.48292864,0.33460729]  #softmax(wk_value) round=3
#wk_ratio =[0.23003816, 0.48038125, 0.28958059]#softmax(wk_value) round=4


In [8]:
round_i

4

In [15]:
parties=2
wk_ratio =  [1] * parties 
wk_ratio = [x / parties for x in wk_ratio]
wk_ratio 


[0.5, 0.5]

In [9]:
#i=2
output_dir='models/vgg16/KAIST/wkAVG'
model_list = avgWeight(model_list,wk_ratio)



save_name = os.path.join(output_dir, 'faster_rcnn_KAIST_AVG_{}_n.pth'.format(round_i))
save_checkpoint({
  #'session': 1,
  'round': round_i,
  'model':  model_list[0].module.state_dict() if args.mGPUs else model_list[0].state_dict(), 
  'optimizer': optimizer.state_dict(),
  'pooling_mode': cfg.POOLING_MODE,
  'class_agnostic': args.class_agnostic,
}, save_name)


In [24]:
wkvalue = [1.945940866,2.682275729,2.176129312]

from scipy.special import softmax
softmax(wkvalue)


array([0.23003816, 0.48038125, 0.28958059])